***GENERATED CODE FOR jointcheck PIPELINE***
***DON'T EDIT THIS CODE***

**READING DATAFRAME**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**PERFORMING OPERATIONS**

In [ ]:


def QueryEditor(spark, joindataobj):
    for source in joindataobj["sourcesForQuery"]:
        sourcename = source['sourcename']
        source['sourcename'] = spark.read.options(header='true', inferschema='true', delimiter=',')\
            .csv('dbfs:' + source['sourcepath'])
        source['sourcename'].createOrReplaceTempView(sourcename)
    joinDf = spark.sql(joindataobj['querytext'])
    df = removeIfDuplicates(joinDf)
    return df


def removeIfDuplicates(joinedDf):
    cols_new = []
    seen = set()
    for c in joinedDf.columns:
        cols_new.append('{}_dup'.format(c) if c in seen else c)
        seen.add(c)
    df = joinedDf.toDF(*cols_new).select(*
                                         [c for c in cols_new if not c.endswith('_dup')])
    return df


**TRANSFORMING DATAFRAME**

In [ ]:
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'price_sum', 'threshold': 212255.76644667142, 'transformation_label': 'Binarizer'}], 'feature': 'price_sum', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '8706', 'mean': '212255.77', 'stddev': '1254140.12', 'min': '70.88', 'max': '1.5953485738516184E7', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}]}, {'feature_label': 'price_sum', 'threshold': 212255.76644667142, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('price_sum')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pprice_sum', 'threshold': 123225.21107927547, 'transformation_label': 'Binarizer'}], 'feature': 'pprice_sum', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '8706', 'mean': '123225.21', 'stddev': '71503.44', 'min': '12913.851499999999', 'max': '448746.4825971811', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}]}, {'feature_label': 'pprice_sum', 'threshold': 123225.21107927547, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('pprice_sum')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**TRAIN MODEL**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run jointcheckHooks.ipynb
try:
			train = DBFSConnector.fetch(spark, "{'url': '/CoupounRedemption/train.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")


		cust = DBFSConnector.fetch(spark, "{'url': '/CoupounRedemption/cust_demo.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")


		coupon = DBFSConnector.fetch(spark, "{'url': '/CoupounRedemption/coupon.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")


		campaign = DBFSConnector.fetch(spark, "{'url': '/CoupounRedemption/campaign.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")


sourcePreExecutionHook()

	transaction = DBFSConnector.fetch(spark, "{'url': '/CoupounRedemption/transactions.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(transaction)

except Exception as ex: 
	logging.error(ex)


**PERFORMING OPERATIONS**

In [ ]:
%run jointcheckHooks.ipynb
try:
	operationPreExecutionHook()

joindata = QueryEditor(spark, {"querytext": "SELECT * \rFROM train\rJOIN coupon \rON train.coupon_id = coupon.coupon_id\rJOIN campaign \rON train.campaign_id = campaign.campaign_id\rJOIN cus\rON train.customer_id = cust.customer_id\rJOIN transaction \rON cust.customer_id = transaction.customer_id", "sourcesForQuery": [{"sourcename": "coupon", "sourcepath": "/CoupounRedemption/coupon.csv", "sourceid": "5eea0c2e1674532b524785c6"}, {"sourcename": "campaign", "sourcepath": "/CoupounRedemption/campaign.csv", "sourceid": "5eea0c041674532b524785c3"}, {"sourcename": "cust", "sourcepath": "/CoupounRedemption/cust_demo.csv", "sourceid": "5eea0c311674532b524785c7"}, {"sourcename": "transaction", "sourcepath": "/CoupounRedemption/transactions.csv", "sourceid": "5eea0c381674532b524785c9"}], "fieldsForQuery": [{"sourcename": "coupon", "fieldname": "coupon_id"}, {"sourcename": "campaign", "fieldname": "campaign_id"}, {"sourcename": "cust", "fieldname": "customer_id"}, {"sourcename": "Train", "fieldname": "id"}, {"sourcename": "Train", "fieldname": "campaign_id"}, {"sourcename": "Train", "fieldname": "coupon_id"}, {"sourcename": "Train", "fieldname": "customer_id"}, {"sourcename": "transaction", "fieldname": "customer_id"}]})
	operationPostExecutionHook(joindata)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run jointcheckHooks.ipynb
try:
	transformationPreExecutionHook()

	jointcheckautofe = TransformationMain.run(joindata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "567.61", "stddev": "327.8", "min": "1", "max": "1115", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "780.76", "stddev": "463.56", "min": "1", "max": "1581", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "redemption_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "1084.49", "stddev": "1370.91", "min": "1", "max": "5395", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "0.2", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "category", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "7.49", "stddev": "2.57", "min": "1", "max": "16", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "-1413.53", "stddev": "7206.46", "min": "-113627.33999999987", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "-2.27", "stddev": "5.7", "min": "-71.24", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_used", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "41.89", "stddev": "206.74", "min": "0.0", "max": "2661.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "item_counts", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "82.6", "stddev": "619.11", "min": "1", "max": "11813", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_customers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "17.43", "stddev": "13.57", "min": "1.0", "max": "118.57142857142857", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "od_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "-34084.94", "stddev": "215152.84", "min": "-3259657.797895498", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "other_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "-20.59", "stddev": "18.38", "min": "-203.7232611506141", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "price_sum", "threshold": 212255.76644667142, "transformation_label": "Binarizer"}], "feature": "price_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "212255.77", "stddev": "1254140.12", "min": "70.88", "max": "1.5953485738516184E7", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qu_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "2555.23", "stddev": "15483.42", "min": "1.0", "max": "207579.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "1.18", "stddev": "0.28", "min": "1.0", "max": "4.096091911309303", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "selling_price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "162.17", "stddev": "139.78", "min": "13.416928746729855", "max": "1235.863616557734", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "t_counts", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "1982.9", "stddev": "11740.96", "min": "1.0", "max": "158788.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "-22.86", "stddev": "19.88", "min": "-203.7232611506141", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "-35498.47", "stddev": "220913.58", "min": "-3344221.267895495", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "0.29", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "44.62", "stddev": "6.36", "min": "32", "max": "63", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age_range", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "2.44", "stddev": "1.24", "min": "0", "max": "5", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "marital_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "0.4", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rented", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "0.06", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "family_size", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "2.23", "stddev": "1.21", "min": "1", "max": "5", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_children", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "0.61", "stddev": "0.97", "min": "0", "max": "3", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "income_bracket", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "4.81", "stddev": "2.34", "min": "1", "max": "12", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "15.22", "stddev": "8.56", "min": "1", "max": "31", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dow", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "3.24", "stddev": "1.69", "min": "0", "max": "6", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_items", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "670.04", "stddev": "333.52", "min": "97", "max": "2088", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "4.9", "stddev": "2.14", "min": "1", "max": "12", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cdd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "-898.21", "stddev": "1522.82", "min": "-11937.800000000007", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "1201.58", "stddev": "671.15", "min": "144", "max": "4134", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "odd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "-16429.2", "stddev": "9899.28", "min": "-62321.37241271824", "max": "-873.1749999999997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qa_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8706", "mean": "180927.78", "stddev": "271252.73", "min": "179", "max": "2617544", "missing": "0"}}, {"transformationsData": [{"feature_label": "pprice_sum", "threshold": 123225.21107927547, "transformation_label": "Binarizer"}], "feature": "pprice_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "8706", "mean": "123225.21", "stddev": "71503.44", "min": "12913.851499999999", "max": "448746.4825971811", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}]}, {"feature": "price_sum_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "8706", "mean": "0.1", "stddev": "0.29", "min": "0.0", "max": "1.0", "missing": "0"}}, {"feature": "pprice_sum_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "8706", "mean": "0.4", "stddev": "0.49", "min": "0.0", "max": "1.0", "missing": "0"}}]}))

	transformationPostExecutionHook(jointcheckautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run jointcheckHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionClassification(jointcheckautofe, ["coupon_id", "customer_id", "brand", "brand_type", "category", "cd_sum", "coupon_discount", "coupon_used", "item_counts", "no_of_customers", "od_sum", "other_discount", "price_sum", "qu_sum", "quantity", "selling_price", "t_counts", "total_discount_mean", "total_discount_sum", "campaign_type", "campaign_duration", "age_range", "marital_status", "rented", "family_size", "no_of_children", "income_bracket", "day", "dow", "no_of_items", "month", "cdd_sum", "customer_id_count", "odd_sum", "qa_sum", "pprice_sum"], "redemption_status")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

